<a href="https://colab.research.google.com/github/fabiobento/multi-view-geometry/blob/master/01_pinhole_camera_calibration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pinhole Camera Model Calibration
This notebook tells you how to calibrate your camera, finding its intrinsic parameters and distortion coefficients
___


## How to do it

1) Import the necessary modules        


In [0]:
import cv2
import numpy as np

The `cv2.imshow()` and `cv.imshow()` functions from the `opencv-python` package are incompatible with Jupyter notebook ([see Stack Overflow issue "cv2.imshow(img) is crashing the kernel"](https://github.com/jupyter/notebook/issues/3935)).
As a replacement, you can use the following function:

In [0]:
from google.colab.patches import cv2_imshow

2) Capture frames from the camera, detect a chessboard pattern on each frame, and accumulate the frames and corners until we have a big enough number of samples:


In [0]:
cap = cv2.VideoCapture(0)

pattern_size = (10, 7)

samples = [ ]

while True:
    ret, frame = cap.read()
    if not ret:
        break

    res, corners = cv2.findChessboardCorners(frame, pattern_size)

    img_show = np.copy(frame)
    cv2.drawChessboardCorners(img_show, pattern_size, corners, res)
    cv2.putText(img_show, 'Samples captured: %d' % len(samples), (0, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)
    cv2_imshow('chessboard', img_show)

    wait_time = 0 if res else 30
    k = cv2.waitKey(wait_time)

    if k == ord('s') and res:
        samples.append((cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), corners))
    elif k == 27:
        break

cap.release()
cv2.destroyAllWindows()

3) Refine all the detected corner points using `cv2.cornerSubPix`:

In [0]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-3)

for i in range(len(samples)):
    img, corners = samples[i]
    corners = cv2.cornerSubPix(img, corners, (10, 10), (-1,-1), criteria)

4) Find the camera's intrinsic parameters by passing all refined corner points to 'cv2.calibrateCamera':

In [0]:
pattern_points = np.zeros((np.prod(pattern_size), 3), np.float32)
pattern_points[:, :2] = np.indices(pattern_size).T.reshape(-1, 2)

images, corners = zip(*samples)

pattern_points = [pattern_points]*len(corners)

rms, camera_matrix, dist_coefs, rvecs, tvecs = \
    cv2.calibrateCamera(pattern_points, corners, images[0].shape, None, None)

np.save('camera_mat.npy', camera_matrix)
np.save('dist_coefs.npy', dist_coefs)

## How it works
Camera calibration aims to find two sets of intrinsic parameters: the camera matrix and distortion coefficients (see [Camera Calibration and 3D Reconstruction](https://docs.opencv.org/2.4/modules/calib3d/doc/camera_calibration_and_3d_reconstruction.html)). The camera matrix determines how coordinates of 3D points are mapped onto dimensionless pixels coordinates in the image, but actual image lenses also distort an image so straight lines are transformed into curves. Distortion coefficients allow you to eliminate such warps.

The whole camera calibration process can be divided into three stages:

* Gathering a decent amount of data such as, images and detected chessboard patterns
* Refining chessboard corners coordinates
* Optimizing camera parameters to match them with observed distortions and projections

To gather data for camera calibration, you need to detect a chessboard pattern of a certain size and accumulate pairs of images and coordinates of the found corners. The function `cv2.findChessboardCorners` implements chessboard corner detection.

It's worth mentioning that corners on the chessboard are the ones formed by two black squares, and the pattern size you pass in `cv2.findChessboardCorners` should be the same as you have in the real chessboard pattern.

The number of samples and their distribution along the field of view is also very important. In practical cases, 50 to 100 samples is enough.

The next step is refining the corner's coordinates. This stage is necessary due to the fact that `cv2.findChessboardCorners` does not give a very accurate result, so we need to find the actual corner positions. `cv2.cornerSubPix` gives precision to corner coordinates with sub-pixel accuracy. It accepts the following arguments:
* Grayscale images
* Coarse coordinates of detected corners
* Size of the refine region to find a more accurate corner position
* Size of the zone in the center of the refine region to ignore
* Criteria to stop the refining process

Coarse coordinates of corners are the ones returned by `cv2.findChessboardCorners`. The refine region should be small, but it should include the actual position of the corner; otherwise, the coarse corner is returned. The size of the zone to ignore should be smaller than the refine region and can be disabled by passing (-1, -1) as its value. The stop criteria can be one of the following types, `cv2.TERM_CRITERIA_EPS` or `cv2.TERM_CRITERIA_MAX_ITER`, or a combinationof the two. `cv2.TERM_CRITERIA_EPS` determines the difference in previous and next corner positions. If the actual difference is less than the one defined, the process will be stopped. `cv2.TERM_CRITERIA_MAX_ITER` determines the maximum number of iterations. `cv2.cornerSubPix` returns the same number of corners with refined coordinates.

Once we have refined the corner's positions, it's time to find the camera's parameters. `cv2.calibrateCamera` solves this problem. You need to pass a few parameters to this function, and these are listed as follows:

* Object points coordinates for all samples
* Corners points coordinates for all samples
* Shape of the images in (width, height) format
* Two arrays to save translation and rotation vectors (can be set to None)
* Flags and stop criteria (both have default values) 

Object points are 3D coordinates of chessboard corners in a chessboard coordinate system. Because we use the same pattern for each frame, the 3D coordinates of the corners are the same and, because we use equidistantly distributed corners, 3D coordinates are also equidistantly distributed on the plane (z=0 for all points). The actual distance between corners doesn't matter because the camera eliminates the z coordinate (how far objects are from the camera), so it can be a smaller but closer pattern, or a bigger but farther one—the image is the same. `cv2.calibrateCamera` returns five values: the mean reprojection error for all samples, the camera matrix, distortion coefficients, rotation, and translation vectors for all samples. The reprojection error is the difference between a corner in the image and the projection of a 3D point of the corner. Ideally, the projection of the corner and its original position in the image should be the same, but there is a difference due to the noise. This difference is measured in pixels. The smaller this difference is, the better the calibration is done. The camera matrix has a shape of 3x3. The number of distortion coefficients depends on the flags and it equals 5 by default.